In [9]:
import pandas as pd
import os
import yfinance as yf
from datetime import datetime, timedelta

# 설정
OUTPUT_DIR = "./macro_data_4h"
MERGED_FILE = os.path.join(OUTPUT_DIR, "eth_4hour.csv")

def rigorous_leakage_check():
    print("🕵️‍♂️ Rigorous Data Leakage & Integrity Check...\n")
    
    if not os.path.exists(MERGED_FILE):
        print("❌ Final dataset not found. Please merge first.")
        return
        
    df = pd.read_csv(MERGED_FILE)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    # S&P 500 컬럼 찾기
    cols = [c for c in df.columns if 'SP500' in c or 'GSPC' in c]
    if not cols:
        print("❌ S&P 500 column not found.")
        return
    sp500_col = cols[0]
    print(f"📊 Using S&P 500 Column: {sp500_col}")
    
    # ---------------------------------------------------------
    # [Test 1] Future Data Leakage (미래 데이터 존재 여부)
    # ---------------------------------------------------------
    now = pd.Timestamp.now()
    last_ts = df['timestamp'].max()
    
    print(f"\n[Test 1] Future Data Check")
    print(f"  - Current System Time: {now}")
    print(f"  - Dataset Last Timestamp: {last_ts}")
    
    if last_ts > now:
        print("  ❌ FAIL: Dataset contains future data! (Leakage)")
    else:
        print("  ✅ PASS: No future data detected.")

    # ---------------------------------------------------------
    # [Test 2] Macro Shift Verification (1일 지연 반영 확인)
    # ---------------------------------------------------------
    print(f"\n[Test 2] Macro Shift Logic Verification (Date: 2023-01-03 US Close)")
    
    try:
        # 1. 실제 정답 데이터 (Yahoo Finance)
        spy = yf.download("^GSPC", start="2023-01-03", end="2023-01-05", progress=False)
        if isinstance(spy.columns, pd.MultiIndex): spy = spy.xs('Close', level=0, axis=1)
        
        real_val_jan3 = spy.loc["2023-01-03"].item() # 1/3 종가 (~3824.14)
        
        print(f"  [Ground Truth]")
        print(f"  - 2023-01-03 US Close: {real_val_jan3:.2f}")
        
        # 2. 내 데이터셋 확인
        # (1) Target: 1월 4일 09:00 (여기에 1/3일 종가가 있어야 정상)
        row_target = df[df['timestamp'] == "2023-01-04 09:00:00"]
        
        # (2) Leak Check: 1월 3일 09:00 (여기에 1/3일 종가가 있으면 누수)
        row_leak = df[df['timestamp'] == "2023-01-03 09:00:00"]
        
        print(f"\n  [My Dataset Check]")
        
        # A. 정상 로직 확인
        if not row_target.empty:
            my_val_target = row_target[sp500_col].values[0]
            print(f"  - Value at 2023-01-04 09:00: {my_val_target:.2f}")
            
            if abs(my_val_target - real_val_jan3) < 0.1:
                print("    -> ✅ PASS: Correctly reflects Jan 3 close.")
            else:
                print(f"    -> ❌ FAIL: Value mismatch (Expected {real_val_jan3:.2f})")
        else:
            print("  ⚠️ Missing target row (2023-01-04 09:00)")

        # B. 누수 확인 (Leakage Check)
        if not row_leak.empty:
            my_val_leak = row_leak[sp500_col].values[0]
            print(f"  - Value at 2023-01-03 09:00: {my_val_leak:.2f}")
            
            if abs(my_val_leak - real_val_jan3) < 0.1:
                print("    -> ❌ FAIL: DATA LEAKAGE! Jan 3 close appeared on Jan 3 morning.")
            else:
                print("    -> ✅ PASS: Value is safe (Different from Jan 3 close).")
        else:
            print("  ⚠️ Missing leak check row (2023-01-03 09:00)")

    except Exception as e:
        print(f"  ⚠️ Error during verification: {e}")

    print("\n" + "="*50)
    print("Verification Complete.")
    print("="*50)

rigorous_leakage_check()


🕵️‍♂️ Rigorous Data Leakage & Integrity Check...

📊 Using S&P 500 Column: SP500

[Test 1] Future Data Check
  - Current System Time: 2025-11-28 22:16:33.924895
  - Dataset Last Timestamp: 2025-11-28 21:00:00
  ✅ PASS: No future data detected.

[Test 2] Macro Shift Logic Verification (Date: 2023-01-03 US Close)


/tmp/ipykernel_1993099/4263920468.py:50: FutureWarning: YF.download() has changed argument auto_adjust default to True
  spy = yf.download("^GSPC", start="2023-01-03", end="2023-01-05", progress=False)


  [Ground Truth]
  - 2023-01-03 US Close: 3824.14

  [My Dataset Check]
  - Value at 2023-01-04 09:00: 3824.14
    -> ✅ PASS: Correctly reflects Jan 3 close.
  - Value at 2023-01-03 09:00: 3839.50
    -> ✅ PASS: Value is safe (Different from Jan 3 close).

Verification Complete.
